In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'
import sys; print(sys.version)

3.8.2 (default, Mar 26 2020, 15:53:00) 
[GCC 7.3.0]


In [2]:
from import_all import *

In [3]:
def save_experiment_dataset(pkl_fn,dev,test):
    dev_len, test_len = len(dev[0]),len(test[0])
    dev_cls_len, test_cls_len = len(set(dev[1])), len(set(test[1]))
    print('N train files: {} | N test files {} | test/ratio: {}'.format(dev_len, test_len, test_len/(dev_len+test_len)))
    print('N train classes: {} | N test classes: {} | test/ratio: {}'.format(dev_cls_len, test_cls_len, test_cls_len/(dev_cls_len+test_cls_len)))
    test_ds = PairDS(test[1],5)
    print('N pairs test: {}'.format(len(test_ds.pairs)))
    save_pickle(pkl_fn,(dev,test,test_ds))
    return dev,test,test_ds

class Experiment:
    def __init__(self, name, *args):
        dev,test,test_ds = save_experiment_dataset(name,*get_elements(*args))
        store_attr('dev,test,test_ds', self)

class Solution:
    def __init__(self, fit, predict): store_attr('fit,predict', self)
        
def test_solution(e,s):
    x1,x2,y = zip(*e.test_ds.pairs)
    x = L(zip(e.test[0][x1], e.test[0][x2]))
    pred=s.predict(x)
    sts = calc_stats(pred,y)
    print_stats(sts)
    return sts

e1 = Experiment('./experiment_1.pkl',
                './data/sigcomp2009', 
                ds_sigcomp2009.get_authentic_signatures, 
                ds_sigcomp2009.get_y,
                lambda fn : 'NISDCC' in fn.name or int(ds_sigcomp2009.get_y(fn)[-3:])<=20)

N train files: 731 | N test files 269 | test/ratio: 0.269
N train classes: 61 | N test classes: 30 | test/ratio: 0.32967032967032966
N pairs test: 2141


In [18]:
def predict(x): return torch.full((len(x),),True)



test_solution(e1,predict)

Accuracy: 0.41802895069122314
Error rate: 0.5819710493087769
False Aceptation Ratio (errores no detectados): 0.0
False Rejection Ratio (falsas alarmas): 0.5819710493087769
Precision: 1.0
Recall (1-FAR): 0.41802895069122314


{'acc': 0.41802895069122314,
 'err': 0.5819710493087769,
 'frr': 0.0,
 'far': 0.5819710493087769,
 'precision': 1.0}

269